# Network and port scanning with Scapy

- replicate some basic features of Nmap and other network scanning tools

In [1]:
! nmap

Nmap 7.91 ( https://nmap.org )
Usage: nmap [Scan Type(s)] [Options] {target specification}
TARGET SPECIFICATION:
  Can pass hostnames, IP addresses, networks, etc.
  Ex: scanme.nmap.org, microsoft.com/24, 192.168.0.1; 10.0.0-255.1-254
  -iL <inputfilename>: Input from list of hosts/networks
  -iR <num hosts>: Choose random targets
  --exclude <host1[,host2][,host3],...>: Exclude hosts/networks
  --excludefile <exclude_file>: Exclude list from file
HOST DISCOVERY:
  -sL: List Scan - simply list targets to scan
  -sn: Ping Scan - disable port scan
  -Pn: Treat all hosts as online -- skip host discovery
  -PS/PA/PU/PY[portlist]: TCP SYN/ACK, UDP or SCTP discovery to given ports
  -PE/PP/PM: ICMP echo, timestamp, and netmask request discovery probes
  -PO[protocol list]: IP Protocol Ping
  -n/-R: Never do DNS resolution/Always resolve [default: sometimes]
  --dns-servers <serv1[,serv2],...>: Specify custom DNS servers
  --system-dns: Use OS's DNS resolver
  --traceroute: Trace hop path to 

In [2]:
# ping scan using nmap - ping scan mac host
! nmap -sn 192.168.195.1

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-27 09:33 MDT
Nmap scan report for 192.168.195.1
Host is up (0.00015s latency).
MAC Address: 00:50:56:C0:00:08 (VMware)
Nmap done: 1 IP address (1 host up) scanned in 0.19 seconds


## ping can be blocked
- disable ping on host and scan again

In [ ]:
! nmap -sn 192.168.195.1

In [3]:
# enable os fingerprinting - may take a few minutes
! nmap -sS -A 192.168.195.1

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-27 09:38 MDT
Nmap scan report for 192.168.195.1
Host is up (0.00037s latency).
Not shown: 965 closed ports, 30 filtered ports
PORT     STATE SERVICE       VERSION
22/tcp   open  ssh           OpenSSH 8.1 (protocol 2.0)
| ssh-hostkey: 
|   2048 07:a6:03:55:10:d0:0a:c3:58:fc:20:c6:55:8f:30:46 (RSA)
|   256 82:69:e5:2a:f4:f0:2f:ed:ae:4e:41:a2:17:a4:cc:64 (ECDSA)
|_  256 aa:26:f4:41:e8:c7:86:fb:b0:61:72:30:dc:a8:cd:de (ED25519)
88/tcp   open  kerberos-sec  Heimdal Kerberos (server time: 2021-10-27 15:38:20Z)
631/tcp  open  ipp           CUPS 2.3
| http-robots.txt: 1 disallowed entry 
|_/
|_http-server-header: CUPS/2.3 IPP/2.1
|_http-title: Home - CUPS 2.3.1
3283/tcp open  netassistant?
5900/tcp open  vnc           Apple remote desktop vnc
| vnc-info: 
|   Protocol version: 3.889
|   Security types: 
|     Apple Remote Desktop (30)
|     Unknown security type (33)
|_    Mac OS X security type (35)
MAC Address: 00:50:56:C0:00:08 (VMware)
Dev

## scapy ping scan
- scan the network with ping/icmp packets and scan for ports on the live hosts

In [2]:
# A system-independent network address manipulation library for Python
! pip install netaddr

    100% |████████████████████████████████| 1.6MB 8.8MB/s eta 0:00:01    78% |█████████████████████████▏      | 1.3MB 6.8MB/s eta 0:00:01


In [3]:
#! /usr/bin/python3

# Adapted for Python 3 from : https://thepacketgeek.com/scapy-p-10-emulating-nmap-functions/

from scapy.all import *
import netaddr
import random

def pingHost(host):
    resp = sr1(IP(dst=str(host))/ICMP(), timeout=2, verbose=0)
    up = False
    #resp.show()
    if not resp:
        print(str(host) + " is down or not responding.")
    elif resp.haslayer(ICMP):
        if (int(resp.getlayer(ICMP).type)==3 and int(resp.getlayer(ICMP).code) in [1,2,3,9,10,13]):
            print(str(host) + " is blocking ICMP.")
        elif (int(resp.getlayer(ICMP).type) == 0): # icmp-response received 
            up = True
        else:
            print("icmp type = {}".format(int(resp.getlayer(ICMP).type)))
            print(str(host) + " is acting weird.")
    return up

# Send ICMP ping request, wait for answer
def pingScanNetwork(addresses):
    liveHosts = []
    for addr in addresses:
        if (addr == addresses.network or addr == addresses.broadcast):
            continue

        print("Trying on {}".format(addr))
        if pingHost(str(addr)):
            liveHosts.append(str(addr))
                
    return liveHosts

## scan a host

In [4]:
host = 'www.facebook.com'
live = pingHost(host)
if live:
    print('{} host is up!'.format(host))

www.facebook.com host is up!


## scan a whole subnet

In [ ]:
# Define IP range to scan
network = "192.168.195.1/24"
# Define TCP port range
portRange = [22,23,80,443,449]
# make list of addresses out of network, set live host counter
addresses = netaddr.IPNetwork(network)
liveHosts = pingScanNetwork(addresses)

Trying on 192.168.47.1
192.168.47.1 is down or not responding.
Trying on 192.168.47.2
192.168.47.2 is down or not responding.
Trying on 192.168.47.3
192.168.47.3 is down or not responding.
Trying on 192.168.47.4
192.168.47.4 is down or not responding.
Trying on 192.168.47.5
192.168.47.5 is down or not responding.
Trying on 192.168.47.6
192.168.47.6 is down or not responding.
Trying on 192.168.47.7
192.168.47.7 is down or not responding.
Trying on 192.168.47.8
192.168.47.8 is down or not responding.
Trying on 192.168.47.9
192.168.47.9 is down or not responding.
Trying on 192.168.47.10
192.168.47.10 is down or not responding.
Trying on 192.168.47.11
192.168.47.11 is down or not responding.
Trying on 192.168.47.12
192.168.47.12 is down or not responding.
Trying on 192.168.47.13
192.168.47.13 is down or not responding.
Trying on 192.168.47.14
192.168.47.14 is down or not responding.
Trying on 192.168.47.15
192.168.47.15 is down or not responding.
Trying on 192.168.47.16
192.168.47.16 is do

## scapy port scanner
- tcp syn scan to map open ports

In [1]:
#! /usr/bin/python3
# Adapted from: https://thepacketgeek.com/scapy-p-10-emulating-nmap-functions/
# Fixed some bugs and ported for Python3

from scapy.all import *
import random
# Define end host and TCP port range

# Send SYN with random Src Port for each Dst port
def scanPorts(host, portRange):
    for dstPort in portRange:
        print("Trying port {:6}".format(dstPort))
        srcPort = random.randint(1025,65534)
        resp = sr1(IP(dst=host)/TCP(sport=srcPort,dport=dstPort,flags="S"),timeout=1,verbose=0)
        if resp:
            if (str(type(resp)) == "<type 'NoneType'>"):
                print(host + ":" + str(dstPort) + " is filtered (silently dropped).")
            elif(resp.haslayer(TCP)):
                if(resp.getlayer(TCP).flags == 0x12):
                    send_rst = sr(IP(dst=host)/TCP(sport=srcPort,dport=dstPort,flags="R"),timeout=1,verbose=0)
                    print (host + ":" + str(dstPort) + " is open.")
            elif (resp.getlayer(TCP).flags == 0x14):
                print (host + ":" + str(dstPort) + " is closed.")
            elif(resp.haslayer(ICMP)):
                if(int(resp.getlayer(ICMP).type)==3 and int(resp.getlayer(ICMP).code) in [1,2,3,9,10,13]):
                    print (host + ":" + str(dstPort) + " is filtered (silently dropped).")
        else:
            print("No response received.")
            

In [2]:
host = "www.facebook.com"
portRange = [22,23,80,443,3389]
scanPorts(host, portRange)

Trying port     22
No response received.
Trying port     23
No response received.
Trying port     80
www.facebook.com:80 is open.
Trying port    443
www.facebook.com:443 is open.
Trying port   3389
No response received.
